# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [4]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_books(min_rating=4.0, max_price=20.0):
    """
    Scrapes book data from Books to Scrape (http://books.toscrape.com/)
    and returns a pandas DataFrame filtered by rating and price.
    
    Parameters:
        min_rating (float): Minimum star rating (1–5)
        max_price (float): Maximum price (£)
    
    Returns:
        pandas.DataFrame: Table with book information
    """
    
    # Base URL
    base_url = "https://books.toscrape.com/catalogue/page-{}.html"
    
    # Mapping from text rating to numerical rating
    rating_map = {
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }
    
    # List to hold all books
    books_data = []
    
    # Loop through all 50 pages on the site
    for page_num in range(1, 51):
        url = base_url.format(page_num)
        response = requests.get(url)
        if response.status_code != 200:
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.find_all('article', class_='product_pod')
        
        for book in books:
            # Title
            title = book.h3.a['title']
            
            # Price (clean up non-breaking spaces and currency symbols)
            price_text = book.find('p', class_='price_color').text
            price_clean = (
                price_text.encode('ascii', 'ignore')  # remove weird encoding characters like Â
                .decode()
                .replace('£', '')
                .strip()
            )
            price = float(price_clean)

            
            # Rating
            rating_class = book.p['class'][1]
            rating = rating_map.get(rating_class, 0)
            
            # Book detail link
            relative_url = book.h3.a['href'].replace('../../../', '')
            book_url = f"https://books.toscrape.com/catalogue/{relative_url}"
            
            # Visit each book's detail page
            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.text, 'html.parser')
            
            # UPC
            upc = book_soup.find('th', text='UPC').find_next('td').text
            
            # Availability
            availability = book_soup.find('p', class_='instock availability').text.strip()
            
            # Description
            desc_tag = book_soup.find('meta', attrs={'name': 'description'})
            description = desc_tag['content'].strip() if desc_tag else "No description available"
            
            # Genre (breadcrumb last but one)
            breadcrumb = book_soup.select('ul.breadcrumb li a')
            genre = breadcrumb[-1].text.strip() if breadcrumb else "Unknown"
            
            # Apply filters
            if rating >= min_rating and price <= max_price:
                books_data.append({
                    "UPC": upc,
                    "Title": title,
                    "Price (£)": price,
                    "Rating": rating,
                    "Genre": genre,
                    "Availability": availability,
                    "Description": description
                })
            
            time.sleep(0.2)  # polite delay between requests

    # Create a DataFrame
    df = pd.DataFrame(books_data)
    
    # Sort by Rating descending, then Price ascending
    df = df.sort_values(by=["Rating", "Price (£)"], ascending=[False, True]).reset_index(drop=True)
    
    return df

# Run the scraper for books rated 4+ and priced <= £20
books_df = scrape_books(min_rating=4.0, max_price=20.0)

# Display the top results
print("Top 10 Highly Rated & Affordable Books:\n")
print(books_df.head(10))

# Optionally export to CSV
books_df.to_csv("highly_rated_books.csv", index=False)
print("\nData saved to 'highly_rated_books.csv'")



/var/folders/d0/51lrq2bs29b460214r0kpfrm0000gn/T/ipykernel_33946/3927533860.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find('th', text='UPC').find_next('td').text


Top 10 Highly Rated & Affordable Books:

                UPC                                              Title  \
0  f36d24c309e87e5b                         An Abundance of Katherines   
1  f201f263d8c23f97                               Greek Mythic History   
2  f701fae56b645ef7  The Power Greens Cookbook: 140 Delicious Super...   
3  94a958ea126cdcd5                                 Dear Mr. Knightley   
4  c4182bdb53e2459a                                The Darkest Corners   
5  924438fc6cbec836  Naturally Lean: 125 Nourishing Gluten-Free, Pl...   
6  c0db71f6bb14db11           Fruits Basket, Vol. 2 (Fruits Basket #2)   
7  02b74068fa98d8a4              Old School (Diary of a Wimpy Kid #10)   
8  ac8e3949d284e9a9  Superman Vol. 1: Before Truth (Superman by Gen...   
9  4fa75fc6829431ca   Every Heart a Doorway (Every Heart A Doorway #1)   

   Price (£)  Rating           Genre             Availability  \
0      10.00       5     Young Adult   In stock (5 available)   
1      10.23  